In [2]:
from functions import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config Completer.use_jedi = False
%matplotlib inline

font = {'size': 15}

In [ ]:
## Import dei dati
#quì importiamo i dati dei sampling in dataframes pandas, la colonna dei numeri d'onda è una unica per tutti
data = pd.read_csv('../data/raw/S1_mapA_11x11.txt', delim_whitespace=True, names=names)
data_bkg = pd.read_csv('../data/raw/S1_bkg_mapA_11x11.txt', delim_whitespace=True, names=names)

In [ ]:
#import degli spettri puri
pure_material_names,pure_materials=import_pure_spectra('../data/raw/Database Raman/BANK_LIST.dat','../data/raw/Database Raman/')

# Explortory data analysis

## Spettri puri
Andiamo prima di tutto a vedere come sono fatti gli spettri dei materiali puri che, come capiamo subito, sono molto meno rumorosi rispetto agli spettri del nostro campione. 

Di seguito plottiamo come esempio lo **spettro puro dell'Albite**. 

Notiamo come lo spettro è poco rumoroso e non presenti offset significativi inoltre lontano dalle frequenze di assorbimento degli stati rotazionali/vibrazionali delle molecole lo spettro va a zero. 



In [ ]:
raman_plot()
plt.plot(pure_materials.Albite_wn,pure_materials.Albite_I)
plt.legend(['Albite']);
plt.rc('font',**font)

Vogliamo ora comparare diversi spettri puri per imparare il più possibile l'andamento delle curve Raman. Notiamo che gli spettri raman dei materiali puri non sono normalizzati tra di loro. 

Dato che l'area sotto lo spettro risulta proporzionale al tempo di esposizione della misura (fonte: pdf di introduzione), questa non è una quantità a cui siamo interessati, scegliamo di normalizzare in modo che tutti gli spettri abbiamo area unitaria. Il seguente è il risultato.

![pure normalized spectrums.](./figures/pure_normalized spectrums.png)

Notiamo che nessuno degli spettri puri presenta un rumore rilevante, assumiamo dunque che questi spettri rappresentino gli spettri Raman corretti per i materiali nella lista.

Assumiamo inoltre che i materiali puri dati coprano quelli di cui sono composti i nostri campioni incogniti (anche se questo non fosse vero è sufficente che coprano i materiali principali).

Osservando il grafico appena plottato deduciamo che:

1. **Per numeri d'onda superiori a 1230 $cm^{-1}$ (linea arancione verticale) tutti gli spettri Raman non presentano picchi rilevanti, l'intensità a piccole lunghezze d'onda è nulla.**
2. **Gli spettri Raman, relativamente alle altezze dei picchi, non presentano un offset rilevante. Nella coda del grafico tutti gli spettri Raman vanno a zero.**
3. **Le FWHM dei picchi sono, certe volte, importanti: fino a 100 $cm^{-1}$.**
4. **Lo spettro fino a circa 1230 $cm^{-1}$ è densamente popolato.**

## Spettri incogniti
### Con cosa stiamo lavorando?
Andiamo a vedere come sono gli **spettri sperimentali**, abbiamo visto che gli spettri con rumore di fondo non contengono informazioni ulteriori rispetto a quelli a background rimosso, dunque non li prendiamo in considerazione. Ne plottiamo alcuni (i primi) e poi tutti per capire con cosa stiamo lavorando.

![first spectrums.](./figures/first_spectrums.png)

![all spectrums.](./figures/all_spectrums.png)

Da questi grafici prendiamo le seguenti scelte nel trattare i dati.

**Rimozione dell'offset**

Come abbiamo visto dagli spettri puri, diversamente dagli spettri incogniti nessuno spettro presentava un offset rilevante. 

Per eliminare/ridurre questo offset utilizziamo il fatto che l'intensita' di tutti gli spettri puri per wave number > 1230 $cm^{-1}$ va a zero.

Imponiamo dunque questo constrain agli spettri incogiti per rimuovere l'offset (calcoliamo l'offset sulla coda e lo sottraiamo a tutto lo spettro). 

**Scegliamo di eliminare la parte destra del grafico**

Questo perchè, come si vede dagli spettri puri, non abbiamo picchi oltre alla frequenza di soglia fissata a $1230 cm^{-1}$.

**Normalizzazione degli spettri del sample**

Normalizziamo in fine i dati per poterli confrontare.

## Risultato dell'EDA
Importiamo i dati elaborati come spiegato (utilizzando make) e vediamo il risultato.

In [ ]:
data = pd.read_csv('../repo_raman/data/EDA_processed_data.dat')
names=define_names() #definisce il nome degli spettri del campione
data.head()

In [ ]:
raman_plot()
plt.rc('font',**font)
for temp in names[1:]:
    plt.plot(data.wn,data_bkg[temp])